# Estudo de caso: 

## Alocação de Incentivos para Usuários e Motoristas da Lyft

A figura abaixo ilustra um exemplo hipotético de um problema de alocação de incentivos: dado um grupo elegível de clientes ou motoristas, um conjunto de incentivos e um orçamento, queremos alocar os incentivos aos clientes ou motoristas para maximizar métricas incrementais, como viagens ou horas de condução.
<img style="float: center;" src="https://miro.medium.com/max/1400/1*mISAbI2U6iU2yNAqj3FxTQ.png">

A alocação de incentivos é um tema comum na Lyft. Por exemplo, a empresa envia cupons promocionais aos clientes e bônus aos motoristas, na esperança de otimizar funções com o objetivo de maximizar o número de viagens ou o número de horas de condução. Nesse contexto, as principais restrições são (1) uma restrição orçamentária total sobre quanto podemos gastar com os incentivos e, às vezes (2) uma exigência de que cada cliente ou motorista possa receber apenas um incentivo no máximo. 

Tais problemas de alocação podem ser facilmente formulados como problemas de **programação inteira**, que podem ser mais relaxados para serem lineares. Embora esses problemas possuam estruturas simples, seu tamanho pode facilmente aumentar (por exemplo, pode haver muitos incentivos ou uma grande base de clientes). Então, nos perguntamos: existe uma maneira rápida e eficiente de resolver esse problema que pode superar os **Solvers** de código-fonte aberto? A resposta é sim. Nesse caso de uso específico, foi desenvolvido um algorítmo baseado na **teoria da dualidade**.

Na Lyft, os cientistas resolvem os mais diversos tipos de problemas de otimização. Embora os **Solvers** possam ser úteis, há momentos em que problemas complicados ou em larga escala precisam ser simplificados e resolvidos com algoritmos personalizados e mais eficientes. Neste caso, veremos como os cientistas da Lyft usaram a dualidade de programação linear para transformar e resolver um problema de alocação de incentivos. Para saber mais detalhes sobre como a Lyft utilizou a **Teoria da Dualidade** de **Programação Linear** para resolver o problema **10x** mais rápido do que utilizando **Solvers** convencionais [veja aqui](https://eng.lyft.com/how-to-solve-a-linear-optimization-problem-on-incentive-allocation-5a8fb5d04db1).

Daqui em diante, você deverá encontrar a solução **Primal** e **Dual** utilizando os **Solver** de Programação Linear Inteira CBC_MIXED_INTEGER_PROGRAMMING.

## Formulação de problema
Por uma questão de simplicidade, vamos considerar um cenário de distribuição de cupons aos clientes apenas. 

**Definimos o seguinte:**
<img style="float: center;" src="https://miro.medium.com/max/1400/1*KrbFUz2QleRSyAYy-0Qhtg.png">

## Primal
<img style="float: center;" src="https://miro.medium.com/max/1400/1*_5K88oTSRTSSKR7fI2uBow.png">

In [1]:
#include <iostream>
#include <random>

In [2]:
#include "setup.h"
#include "ortools/linear_solver/linear_solver.h"
#include "ortools/linear_solver/linear_solver.pb.h"

using namespace operations_research;

## Crie uma variável para o cliente (rider) i

In [3]:
int i;

## Crie uma variável para o cupom do tipo j

In [4]:
int j;

## Crie uma constante para o total de m clientes (riders), use m=100

In [5]:
const int m = 100;

## Crie uma constante para o número máximo de tipos de cupons distintos, use k=10

In [6]:
const int k = 10;

## Para cada cliente (rider) i = 1,...,m, há $k_i$ tipos distintos de cupons que podem ser atribuídos a cada cliente. Observe que $k_i$ pode ser diferente para cada cliente.

In [7]:
std::vector<int> k_i (m);

## Defina $x_{ij}$, onde i=1,...,m; e j=1,...,$k_i$, como uma variável binária indicando se o cliente i recebe um cupom do tipo j. Utilize vetores para armazenamento.

In [8]:
std::vector<std::vector<const MPVariable*>> x(m, std::vector<const MPVariable*>(k));

## Para cada cliente (rider) i e tipo de cupom j, há um valor associado $v_{ij}$, onde i=1,...,m; e j=1,...,$k_i$. Utilize vetores para armazenamento.

In [9]:
std::vector<std::vector<int>> v (m, std::vector<int>(k));

## Para cada cliente (rider) i e tipo de cupom j, há um custo associado $c_{ij}$, onde i=1,...,m; e j=1,...,$k_i$.

In [10]:
std::vector<std::vector<int>> c (m, std::vector<int>(k));

## Observe nos dados aleatórios gerados abaixo que alguns clientes (riders) não têm todos os tipos de incentivos k, mas apenas os primeiros $k_i$

In [11]:
std::default_random_engine generator;  
std::uniform_int_distribution<int> uniform_dist(0, 100);
std::uniform_int_distribution<int> incentives_dist(1, k);

for(i=0; i < m; i++) {
    k_i[i] = incentives_dist(generator);
    for(j=0; j < k; j++) {
        v[i][j] = uniform_dist(generator);
        c[i][j] = uniform_dist(generator);
    }
}

## Considere um orçamento C, que limita o custo total que pode ser incorrido em uma tarefa viável. $C = \frac{\sum c_{ij}}{2k}$

In [12]:
int C (0);
for(i=0; i < m; i++) 
    for(j=0; j < k; j++) 
        C += c[i][j];
C /= 2 * k;

## Crie o resolvedor MIP com o back-end CBC.

In [13]:
MPSolver solver("simple_mip_program", MPSolver::CBC_MIXED_INTEGER_PROGRAMMING);

## Defina as combinações de variáveis $x_{ij}$ que não possuam todos os tipos de cupons disponívels com limite superior 0 para eliminá-las do espaço de solução. Isso pode ser usado pelo solucionador na etapa de pré-resolução, reduzindo o tamanho do modelo. As variáveis que possuem cupons disponíveis são aquelas que $k_i < k$ e, nesse caso, o limite inferior deve ser 1.

In [14]:
int count(0);
for(i=0; i < m; i++) {
    for(j=0; j < k; j++) {
        double ub = j < k_i[i] ? 1.0 : 0.0;
        x[i][j] = solver.MakeIntVar(0.0, ub, "");
        count += ub;
    }
}
std::cout << "Number of variables = " << count << std::endl;

Number of variables = 579


## Crie a restrição de orçamento usando o C calculado acima

In [15]:
MPConstraint* budget = solver.MakeRowConstraint(0, C, "budget");
for (i = 0; i < m; i++) 
    for (j = 0; j < k_i[i]; j++) 
      budget->SetCoefficient(x[i][j], c[i][j]);

std::cout << "Number of constraints = " << solver.NumConstraints() << std::endl;

Number of constraints = 1


## Crie restrições para que cada cliente possa ser receber no máximo um cupom ao todo.

In [16]:
for (i = 0; i < m; i++) {
    MPConstraint* constraint = solver.MakeRowConstraint(0, 1, "");
    for (j = 0; j < k_i[i]; j++) 
        constraint->SetCoefficient(x[i][j], 1.0);
}
std::cout << "Number of constraints = " << solver.NumConstraints() << std::endl;

Number of constraints = 101


## Crie a função objetivo

In [17]:
MPObjective* objective = solver.MutableObjective();
for (i = 0; i < m; i++) 
    for (j = 0; j < k_i[i]; j++)
        objective->SetCoefficient(x[i][j], v[i][j]);

## O objetivo é maximizar o valor total dos cupons atribuídos.

In [18]:
objective->SetMaximization();

## Verifique se o problema tem uma solução ótima e imprima seu valor.

In [19]:
const MPSolver::ResultStatus result_status = solver.Solve();

if (result_status != MPSolver::OPTIMAL) 
    std::cout << "The problem does not have an optimal solution." << std::endl;

std::cout << "Optimal objective value = " << objective->Value() << std::endl;


Optimal objective value = 7321


## Monte um histograma numa tabela 10x1 com todos os cupons distribuidos k. Conte o número de vezes que $x_{ij}==1$ para cada j

In [20]:
{
    std::vector<int> histograma (10, 0);

    for (i = 0; i < m; i++)
       for (j = 0; j < k; j++)
           if (x[i][j]->solution_value())
               histograma[j]++;

    int total(0);
    for (j = 0; j < k; j++) {
        total += histograma[j];
        std::cout << "Cupom " << j+1 << " : " << histograma[j] << std::endl;
    }

    std::cout << "Total de Cupons:  " << total << std ::endl;
}

Cupom 1 : 24
Cupom 2 : 19
Cupom 3 : 15
Cupom 4 : 11
Cupom 5 : 13
Cupom 6 : 6
Cupom 7 : 1
Cupom 8 : 0
Cupom 9 : 4
Cupom 10 : 0
Total de Cupons:  93


## Verifique se os dados resultantes satisfazem as restrições (custo <= C)

In [21]:
{
    int custo(0);
    for (i = 0; i < m; i++)
        for (j = 0; j < k; j++)
            custo += c[i][j] * x[i][j]->solution_value(); 

    if(custo <= C)
        std::cout<< custo << " <= " << C << std::endl;
    else
        std::cout<< custo << " > " << C << std::endl;
}

2485 <= 2485 Ok 


## Verifique se o valor total da resposta é igual à função objetivo

In [22]:
{
    int resposta(0);
    
    for (i = 0; i < m; i++)
        for (j = 0; j < k; j++)
            resposta += v[i][j]*x[i][j]->solution_value();

    if (resposta != objective->Value())
        std::cout << "Valor total da resposta (" << resposta << ") é diferente da FO. " << objective->Value() << std::endl;
    else
        std::cout << "Valor total da resposta é igual a FO. " << resposta << std::endl;
}

Valor total da resposta igual que a funcao objetivo. 7321


## Verifique se cada cliente recebeu apenas um cupom no máximo

In [23]:
{
    for (i = 0; i < m; i++){
        int cupons(0);
        
        for (j = 0; j < k; j++)
           if(x[i][j]->solution_value() == 1.0)
                cupons++;
        
        if(cupons > 1)
            std::cout<< "Cliente " << i+1 << " recebeu mais de um cupom!" << std::endl;
    }
}

**Definimos o seguinte:**
<img style="float: center;" src="https://miro.medium.com/max/1400/1*ES8QjWf5Xe_v5mprEiRELQ.png">

## Dual
<img style="float: center;" src="https://miro.medium.com/max/1400/1*nWbaZVpT_fjXDQYDo7P2Tg.png">

In [24]:
{
    MPSolver solver("simple_mip_program", MPSolver::CBC_MIXED_INTEGER_PROGRAMMING);

    const double infinity = solver.infinity();

    std::vector<const MPVariable*> y_i(m);

    for (i = 0; i < m; i++)
        y_i[i] = solver.MakeIntVar(0.0, infinity, "");

    auto const lambda = solver.MakeNumVar(0.0, infinity, "");

    std::cout << "Número de variáveis = " << solver.NumVariables() << std::endl;

    for (i = 0; i < m; i++){
        for(j = 0; j < k_i[i]; j++){
            auto const c1 = solver.MakeRowConstraint(v[i][j], infinity, "");
            c1->SetCoefficient(y_i[i], 1.0);
            c1->SetCoefficient(lambda, c[i][j]);
        }
    }

    std::cout << "Número de restrições = " << solver.NumConstraints() << std::endl;

    auto const objetivo = solver.MutableObjective();
    for (i = 0; i < m; i++)
            objetivo->SetCoefficient(y_i[i], 1.0);
    objetivo->SetCoefficient(lambda, C);

    objetivo->SetMinimization();

    auto const resultado_status = solver.Solve();
    
    if (resultado_status == MPSolver::OPTIMAL) {
        std::cout << "O problema tem uma solucao otima!" << std::endl;
        std::cout << "Objective value = " << objetivo->Value() << std::endl;      
    }
    else{
        std::cout << "O problema nao tem uma solucao otima!";  
    }
}

Número de variáveis = 101
Número de restrições = 579
O problema tem uma solucao otima!
Objective value = 7340
